# মডেল কনটেক্সট প্রোটোকল (MCP) সমর্থন সহ Azure AI এজেন্ট

এই নোটবুকটি দেখায় কীভাবে মডেল কনটেক্সট প্রোটোকল (MCP) টুল ব্যবহার করে Azure AI এজেন্ট তৈরি করা যায়, যা উন্নত ক্ষমতার জন্য বাহ্যিক MCP সার্ভারগুলোর সুবিধা নিতে পারে।


## প্রয়োজনীয় NuGet প্যাকেজ ইনস্টল করুন

প্রথমে, আমাদের Azure AI Agents Persistent প্যাকেজটি ইনস্টল করতে হবে যা Azure AI Agents-এর সাথে কাজ করার জন্য মূল কার্যকারিতা প্রদান করে।


## কীলেস অথেন্টিকেশন সুবিধা

এই নোটবুকটি **কীলেস অথেন্টিকেশন** প্রদর্শন করে, যা বেশ কয়েকটি সুবিধা প্রদান করে:
- ✅ **কোনো API কী পরিচালনা করার প্রয়োজন নেই** - Azure পরিচয়-ভিত্তিক অথেন্টিকেশন ব্যবহার করে
- ✅ **উন্নত নিরাপত্তা** - কোড বা কনফিগারেশনে কোনো সিক্রেট সংরক্ষণ করা হয় না
- ✅ **স্বয়ংক্রিয় ক্রেডেনশিয়াল রোটেশন** - Azure ক্রেডেনশিয়ালের জীবনচক্র পরিচালনা করে
- ✅ **রোল-ভিত্তিক অ্যাক্সেস** - সূক্ষ্ম-স্তরের অনুমতির জন্য Azure RBAC ব্যবহার করে

`DefaultAzureCredential` স্বয়ংক্রিয়ভাবে সেরা উপলব্ধ ক্রেডেনশিয়াল উৎস ব্যবহার করবে:
1. ম্যানেজড আইডেন্টিটি (যখন Azure-এ চলছে)
2. Azure CLI ক্রেডেনশিয়াল (উন্নয়নের সময়)
3. Visual Studio ক্রেডেনশিয়াল
4. পরিবেশ ভেরিয়েবল (যদি কনফিগার করা থাকে)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Azure পরিষেবাগুলির সাথে প্রমাণীকরণের জন্য DefaultAzureCredential ব্যবহার করে Azure Identity প্যাকেজ ইনস্টল করুন।


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## প্রয়োজনীয় নেমস্পেস আমদানি করুন

Azure AI Agents এবং Azure Identity এর জন্য প্রয়োজনীয় নেমস্পেস আমদানি করুন।


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Azure AI Agent Client (কী ছাড়াই প্রমাণীকরণ) কনফিগার করুন

কনফিগারেশন ভেরিয়েবল সেট আপ করুন এবং **কী ছাড়াই প্রমাণীকরণ** ব্যবহার করে PersistentAgentsClient তৈরি করুন:
- **projectEndpoint**: Azure AI Foundry প্রকল্পের এন্ডপয়েন্ট
- **modelDeploymentName**: ডিপ্লয় করা AI মডেলের নাম (GPT-4.1 nano)
- **mcpServerUrl**: MCP সার্ভারের URL (Microsoft Learn API)
- **mcpServerLabel**: MCP সার্ভার সনাক্ত করার জন্য একটি লেবেল
- **DefaultAzureCredential**: ম্যানেজড আইডেন্টিটি, Azure CLI, বা অন্যান্য প্রমাণীকরণ উৎস ব্যবহার করে (API কী প্রয়োজন নেই)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## MCP টুল ডেফিনিশন তৈরি করুন

একটি MCP টুল ডেফিনিশন তৈরি করুন যা Microsoft Learn MCP সার্ভারের সাথে সংযুক্ত হবে। এটি এজেন্টকে Microsoft Learn-এর কন্টেন্ট এবং ডকুমেন্টেশনে প্রবেশের সুযোগ দেবে।


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## এআই এজেন্ট তৈরি করুন

নির্দিষ্ট মডেল এবং MCP টুল ব্যবহার করে একটি স্থায়ী এআই এজেন্ট তৈরি করুন। এজেন্টটি নিম্নলিখিতভাবে কনফিগার করা হয়েছে:
- GPT-4.1 ন্যানো মডেল
- MCP টুল ব্যবহার করার নির্দেশনা
- Microsoft Learn MCP সার্ভারে অ্যাক্সেস


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## থ্রেড তৈরি করুন এবং বার্তা পাঠান

একটি কথোপকথনের থ্রেড তৈরি করুন এবং ব্যবহারকারীর কাছে একটি বার্তা পাঠান যেখানে Azure OpenAI এবং OpenAI-এর পার্থক্য সম্পর্কে জিজ্ঞাসা করা হয়েছে। এটি এজেন্টের MCP টুল ব্যবহার করে সঠিক তথ্য প্রদান করার দক্ষতা পরীক্ষা করবে।


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## MCP টুল রিসোর্স কনফিগার করুন (কী ছাড়া)

MCP টুল রিসোর্স সেট আপ করুন। সম্পূর্ণ কী ছাড়া পদ্ধতির জন্য, আপনি কাস্টম হেডার সরিয়ে দিতে পারেন যদি MCP সার্ভার Azure পরিচয়-ভিত্তিক প্রমাণীকরণ সমর্থন করে। নিচের উদাহরণটি দেখায় কীভাবে প্রয়োজন হলে হেডার যোগ করা যায়, তবে কী ছাড়া পরিস্থিতিতে এগুলো প্রয়োজন নাও হতে পারে।


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## এজেন্ট রান শুরু করুন

ব্যবহারকারীর বার্তা প্রক্রিয়া করার জন্য একটি রান তৈরি করুন এবং শুরু করুন। এজেন্ট কনফিগার করা MCP টুল এবং রিসোর্স ব্যবহার করে একটি প্রতিক্রিয়া তৈরি করবে।


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## রান পর্যবেক্ষণ করুন এবং টুল অনুমোদন পরিচালনা করুন (কীলেস)

এজেন্টের রান স্ট্যাটাস পর্যবেক্ষণ করুন এবং প্রয়োজনীয় টুল অনুমোদন পরিচালনা করুন। এই লুপটি:
1. রান সম্পন্ন হওয়া বা কোনো পদক্ষেপের প্রয়োজন হলে অপেক্ষা করে
2. MCP টুল কল স্বয়ংক্রিয়ভাবে অনুমোদন করে যখন প্রয়োজন হয়
3. কীলেস প্রমাণীকরণের ক্ষেত্রে, যদি MCP সার্ভার Azure পরিচয় সমর্থন করে তবে হেডার প্রয়োজন নাও হতে পারে


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## কথোপকথনের ফলাফল প্রদর্শন করুন

থ্রেডের সমস্ত বার্তা পুনরুদ্ধার করুন এবং প্রদর্শন করুন, যেখানে ব্যবহারকারীর প্রশ্ন এবং এজেন্টের উত্তর উভয়ই দেখানো হবে। বার্তাগুলি কালানুক্রমিক ক্রমে টাইমস্ট্যাম্প এবং ভূমিকা সূচক সহ প্রদর্শিত হয়।


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিকতার জন্য চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা তার জন্য দায়ী থাকব না।
